<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/FlexinMarch23.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Downloading required packages
!pip install yfinance
!pip install numpy
!pip install pandas
!pip install hurst
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import of the necessary modules
from IPython.display import clear_output 
from time import sleep
from tqdm.notebook import tqdm
import pandas as pd
import requests

In [3]:
def send_to_telegram(message):

    apiToken = '6183952601:AAFbfrKfZb3iKX6GfYZQnatxo0lg1S4lL8Y'
    chatID = '@flexinflexinbot1'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)

In [7]:
class Flexin:
  
  def __init__(self, wallet):
    '''
    Symbol of any pair, stock or commodity
    :param symbol: symbol
    '''
    self.wallet = wallet

  # Objects
  def ok(self):
    # Packages needed for this function
    import yfinance as yf
    from hurst import compute_Hc
    '''
    Returns data for any symbol - The first collection is made for the daily
    chart and then the hourly chart.
    :param symbol: asset symbol for analysis 
    '''
    for symbol in wallet:
      # BBs variables
      n = 20 # Moving average periods
      k = 2 # Factor to shift the bands
      print(f'analyzing {symbol}...')
      #------------------------------------------------------------------------- 
      # Daily filter - For period => 'max' and interval='1d'
      #-------------------------------------------------------------------------
      d = yf.Ticker(symbol).history(period='max', interval='1d')
      #d.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
      # Hurst exponent filter (https://github.com/Mottl/hurst)
      H = lambda x: compute_Hc(x)[0]
      d['Hurst'] = d['Close'].rolling(window=1024).apply(H)
      # Bollinger Bands
      d['Standard Deviation'] = d['Close'].rolling(n).std()
      d['Middle Band'] = d['Close'].rolling(n).mean()
      d['Upper Band'] = d['Middle Band'] + d['Standard Deviation']*k
      d['Lower Band'] = d['Middle Band'] - d['Standard Deviation']*k
      # OK
      print('🆗 Step 1 - Successful Daily data collection!')
      #-------------------------------------------------------------------------
      # Hour filter - For period => '1y' and interval='1h'
      #-------------------------------------------------------------------------
      h = yf.Ticker(symbol).history(period='1y', interval='1h')
      #h.drop(['Volume', 'Dividends', 'Stock Splits'], axis=1, inplace=True)
      # Hurst exponent filter (https://github.com/Mottl/hurst)
      H = lambda x: compute_Hc(x)[0]
      h['Hurst'] = h['Close'].rolling(window=1024).apply(H)
      # Bollinger Bands
      h['Standard Deviation'] = h['Close'].rolling(n).std()
      h['Middle Band'] = h['Close'].rolling(n).mean()
      h['Upper Band'] = h['Middle Band'] + h['Standard Deviation']*k
      h['Lower Band'] = h['Middle Band'] - h['Standard Deviation']*k
      # OK
      print('🆗 Step 2 - Successful Hour data collection!')
      #-------------------------------------------------------------------------
      # Strategy
      #-------------------------------------------------------------------------
      verify_long = False
      verify_short = False
      Hurst = h['Hurst'][-1]
      #-------------------------------------------------------------------------
      if 1>d['Hurst'][-1]>0.5 and 1>h['Hurst'][-1]>0.5:
        print(f'>>> for {symbol} 🏂! H© = {Hurst.round(4)} with persistent behavior.')
        if d['Close'][-1]>d['Middle Band'][-1] and d['Open'][-1]>d['Middle Band'][-1] and h['Close'][-1]>h['Upper Band'][-1] and h['Open'][-1]>h['Middle Band'][-1]: # BUY/LONG
          verify_long = True
          print(f'>>> 📈 BUY: {verify_long}.')
        else:
          if d['Close'][-1]<d['Middle Band'][-1] and d['Open'][-1]<d['Middle Band'][-1] and h['Close'][-1]<h['Lower Band'][-1] and h['Open'][-1]<h['Middle Band'][-1]: # SELL/SHORT
            verify_short = True
            print(f'📉 SELL: {verify_short}.')
      else:
        print(f'>>> for {symbol} -> H© = {Hurst.round(4)} 💤! Anti-persistent behavior.')
      #-------------------------------------------------------------------------
      print('')
    #---------------------------------------------------------------------------
  #-----------------------------------------------------------------------------

In [5]:
wallet = ['EURUSD=X',
          'JPY=X',
          'GBPUSD=X',
          'CHF=X',
          'AUDUSD=X',
          'CAD=X',
          'NZDUSD=X',
          '^HSI']

In [ ]:
while True:
  Flexin(wallet).ok()
  clear_output()